### 1 часть – общий пример

Цель: научиться использовать библиотеку LightFM для построения гибридной
рекомендательной системы, а также оценить качество полученной модели на
реальных данных.

1. Установить библиотеку LightFM


In [1]:
!pip install git+https://github.com/daviddavo/lightfm

  Cloning https://github.com/daviddavo/lightfm to /tmp/pip-req-build-wiim353c
  Running command git clone --filter=blob:none --quiet https://github.com/daviddavo/lightfm /tmp/pip-req-build-wiim353c
  Resolved https://github.com/daviddavo/lightfm to commit f0eb500ead54ab65eb8e1b3890337a7223a35114
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=1070682 sha256=ff65e4b4e92b4c52638e78d674200dc4655b7353911356f1ed021a7c9e83be35
  Stored in directory: /tmp/pip-ephem-wheel-cache-gzmoklyy/wheels/c8/8f/1d/d3f463d8a4f3c02329387523b4a67d9a415ba7776580d7eee2
Successfully built lightfm


2. Выполнить импорт необходимых библиотек


In [2]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, auc_score

3. Подготовка данных\
Функция fetch_movielens загружает набор данных Movielens и возвращает
словарь, содержащий матрицу оценок, пользовательские функции,
характеристики элементов и другую информацию. В построении данной рек.
системы нас интересует только матрица оценок.


In [ ]:
data = fetch_movielens(min_rating=5.0)

4. Реализация гибридной рекомендательной системы\
Для реализации рекомендательной системы с lightFM предпочтительным
алгоритмом является WARP (Weighted Approximate-Rank Pairwise).

In [ ]:
model = LightFM(loss='warp')
model.fit(data['train'],
          epochs=30,
          num_threads=2,
          verbose=True)

Epoch: 100%|██████████| 30/30 [00:05<00:00,  5.21it/s]


5. Генерация рекомендаций\
Используем созданную модель для генерации рекомендаций для
пользователей, вызвав метод recommend. Метод recommend принимает два
аргумента: user_ids и item_ids . Мы можем генерировать рекомендации для
одного пользователя или нескольких пользователей одновременно. Мы также
можем указать количество рекомендаций для генерации. Следующий код
генерирует 10 рекомендаций для пользователя 3.

In [ ]:
import numpy as np
user_id = 3
n_items = data['train'].shape[1]
recommendations = model.predict(user_id, np.arange(n_items))
top_items = np.argsort(-recommendations)[:10]
top_items

array([299, 325, 301, 878, 345, 269, 312, 326, 358, 321])

6. Оценка рекомендательной системы\
Оцените производительность модели, используя метрики precision at k и
AUC. Метрика precision at k измеряет процент рекомендаций, которые были
релевантны пользователю, из числа k лучших рекомендаций. Метрика AUC
измеряет площадь под кривой рабочей характеристики приемника (ROC),
которая показывает частоту истинных положительных срабатываний по
сравнению с частотой ложных положительных срабатываний.\
Мы можем использовать библиотеку LightFM для вычисления этих
показателей. Точность из k рекомендаций может быть рассчитана с помощью
метода precision_at_k, а AUC может быть рассчитан с помощью метода auc_score.\
В этом примере мы обучаем модель на основе матрицы взаимодействий и
оцениваем точность по метрикам k и AUC как для обучающего, так и для
тестового наборов данных. Мы используем значение 10 для k, что означает, что
мы рассматриваем только 10 лучших рекомендаций для каждого пользователя.
Можно также попробовать вариант модели для k = 5.

In [ ]:
train_precision = np.mean(precision_at_k(model, data['train'], k=10, num_threads=2))
train_auc = np.mean(auc_score(model, data['train'], num_threads=2))
test_precision = np.mean(precision_at_k(model, data['test'], k=10, num_threads=2))
test_auc = np.mean(auc_score(model, data['test'], num_threads=2))
print('Train precision: {:.2f}'.format(train_precision))
print('Train AUC: {:.2f}'.format(train_auc))
print('Test precision: {:.2f}'.format(test_precision))
print('Test AUC: {:.2f}'.format(test_auc))

Train precision: 0.36
Train AUC: 0.97
Test precision: 0.05
Test AUC: 0.91


### 2 часть – ИНДЗ
Тема: Реализация гибридной модели рекомендаций на собственных данных
Цель: научиться использовать библиотеку LightFM для построения гибридной
рекомендательной системы, которая комбинирует коллаборативную
фильтрацию и учёт признаков пользователей и/или товаров, на вашем
собственном наборе данных.

Требования к данным
Ваш набор данных должен содержать:
1. Матрицу взаимодействий (user-item matrix):
- Строки — пользователи, столбцы — товары, значения — оценки или
факты взаимодействий (например, клики, покупки).
- Матрица может быть разреженной, например, в формате CSR
(Compressed Sparse Row).
2. Признаки пользователей (User Features):
- Пример: возраст, пол, местоположение, категории интересов.
- Если таких данных нет, вы можете использовать только
идентификаторы пользователей (в виде one-hot encoding).
3. Признаки товаров (Item Features):
- Пример: жанры фильмов, категории продуктов, цена, бренд.
- Если таких данных нет, используйте только идентификаторы
товаров.

In [ ]:
import json, time
from collections import defaultdict
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score
from scipy.sparse import lil_matrix
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

1. Подготовка данных:

Загрузите ваш набор данных и подготовьте три компонента:
- матрицу взаимодействий;
- признаки пользователей;
- признаки товаров.

Если у вас нет явных признаков пользователей или товаров,
используйте dummy-признаки (например, идентификаторы).

In [ ]:
review_path = '/content/AMAZON_FASHION_5.json'
interactions = []
seen = set()
user_raw = defaultdict(set)
item_raw = defaultdict(set)
item_summary = defaultdict(list)

with open(review_path, 'r', encoding='utf-8') as f:
    for line in f:
        r = json.loads(line)
        if r.get('overall', 0) < 5.0:
            continue
        uid = r['reviewerID']
        iid = r['asin']
        if (uid, iid) in seen:
            continue
        seen.add((uid, iid))
        interactions.append((uid, iid))
        if r.get('verified'):
            user_raw[uid].add('verified')
        year = time.gmtime(r['unixReviewTime']).tm_year
        user_raw[uid].add(f'year:{year}')
        length = len(r.get('reviewText', '').split())
        user_raw[uid].add(f'revlen_bucket:{length//50}')
        item_summary[iid].append(r.get('summary', ''))
        for k, v in r.get('style', {}).items():
            if v:
                item_raw[iid].add(f"{k.strip().lower()}{v.strip()}")
                
item_ids = list(item_summary.keys())
docs = [' '.join(item_summary[iid]) for iid in item_ids]
vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(docs)

all_user_feats = {f for fs in user_raw.values() for f in fs}
all_item_feats = {f for fs in item_raw.values() for f in fs}
dataset = Dataset()
dataset.fit((u for u, _ in interactions),
            (i for _, i in interactions),
            user_features=all_user_feats,
            item_features=all_item_feats)
interactions_matrix, _ = dataset.build_interactions(interactions)
uf = [(u, list(fs)) for u, fs in user_raw.items()]
if_ = [(i, list(fs)) for i, fs in item_raw.items()]
user_features_matrix = dataset.build_user_features(uf, normalize=True)
item_features_matrix = dataset.build_item_features(if_, normalize=True)
n_users, n_items = interactions_matrix.shape
item_features_combined = hstack([item_features_matrix, tfidf_matrix], format='csr')

2. Обучение модели:
- Используйте алгоритм WARP (Weighted Approximate-Rank Pairwise)
для обучения модели.
- Учтите, что LightFM принимает признаки пользователей и товаров в
формате разреженных матриц.

In [ ]:
test_ratio = 0.2
train = interactions_matrix.tolil()
test = lil_matrix(train.shape, dtype=np.int32)
rng = np.random.default_rng(42)

for u in range(n_users):
    pos = train.rows[u]
    if len(pos) < 2:
        continue
    n_test = max(1, int(len(pos) * test_ratio))
    for i in rng.choice(pos, size=n_test, replace=False):
        train[u, i] = 0
        test[u, i] = 1

train = train.tocsr()
test = test.tocsr()

model = LightFM(loss='warp', no_components=64, learning_rate=0.03)
model.fit(train,
          user_features=user_features_matrix,
          item_features=item_features_combined,
          epochs=50,
          num_threads=1,
          verbose=True)

"Epoch: 100%|██████████| 50/50 [00:41<00:00,  1.20it/s]



3. Генерация рекомендаций:
- Создайте список топ-N рекомендаций для нескольких
пользователей.
- Для проверки выберите хотя бы 5 пользователей

In [ ]:
N = 10
all_items = np.arange(n_items, dtype=np.int32)

for uid in range(min(n_users, 5)):
    scores = model.predict(user_ids=np.full(n_items, uid, dtype=np.int32),
                           item_ids=all_items,
                           user_features=user_features_matrix,
                           item_features=item_features_combined,
                           num_threads=1)
    scores[train[uid].indices] = -np.inf
    top_items = np.argsort(-scores)[:N]
    print(f"Top-{N} для user {uid}: {top_items}")

Top‑10 для user 0: [21 20 13 10 11  8 14 12 18 16]
Top‑10 для user 1: [21 20 13 10 11 14  8 12 18 16]
Top‑10 для user 2: [21 20 13 10 14 11  8 12 18 15]
Top‑10 для user 3: [21 20 13 10 14 12 11  8  0 18]
Top‑10 для user 4: [21 20 13 10  8 11 14 12 16 18]


4. Оценка модели:
- Оцените производительность модели, используя метрики
precision@k и AUC.
- Проверьте результаты как на обучающем, так и на тестовом наборах
данных.

In [ ]:
valid_users = np.where(test.getnnz(axis=1) > 0)[0]
if len(valid_users) > 0:
    users = rng.choice(valid_users, size=len(valid_users), replace=False)
    train_sub = train[users]
    test_sub = test[users]
    user_feat_sub = user_features_matrix[users]

    train_prec = precision_at_k(model, train_sub, k=5,
                                user_features=user_feat_sub,
                                item_features=item_features_combined).mean()
    train_auc = auc_score(model, train_sub,
                          user_features=user_feat_sub,
                          item_features=item_features_combined).mean()
    test_prec = precision_at_k(model, test_sub, train_interactions=train_sub, k=5,
                               user_features=user_feat_sub,
                               item_features=item_features_combined,
                               check_intersections=False).mean()
    test_auc = auc_score(model, test_sub, train_interactions=train_sub,
                         user_features=user_feat_sub,
                         item_features=item_features_combined,
                         check_intersections=False).mean()

    print(f"train p@5={train_prec:.2f}  AUC={train_auc:.2f}")
    print(f"test p@5={test_prec:.2f}  AUC={test_auc:.2f}")

train p@5=0.96  AUC=0.99
test p@5=0.20  AUC=0.99
